## Gridding and optimal interpolation

### Data: 
- Monterey Bay Glider line [IOOS Glider DAC](https://gliders.ioos.us/map/#)

### Gridding and plotting tools:
- [Scipy griddata function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html)
- [Glidertools kriging](https://glidertools.readthedocs.io/en/latest/mapping.html)


In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cmocean.cm as cmo

from scipy.interpolate import griddata
import glidertools as gt

## Loading and plotting the data

In [ ]:
csv_glider = 'ioos_glider/sp064-20240130T2048_004f_97f4_cbff.csv'
df = pd.read_csv(csv_glider, skiprows=[1], parse_dates=[3])

In [ ]:
df

Plot the entire dissolved oxygen dataset

#### Exercises

- Label the plot above and add a colorbar
- Make a plot of time vs. longitude
- Identify a time range where the glider conducts one complete transect

### Choosing a subset

In [ ]:
t1_str = # insert string
t2_str = # insert string

t1 = pd.Timestamp(t1_str, tz='UTC')
t2 = pd.Timestamp(t2_str, tz='UTC')

time_mask = (df['time'] >= t1) & (df['time'] < t2)

dfsub = df.loc[time_mask].dropna(subset=['dissolved_oxygen', 'depth', 'longitude'])

In [ ]:
plt.figure()
plt.subplot(3,1,1)
plt.scatter(dfsub['longitude'], dfsub['depth'], c=dfsub['temperature'], cmap=cmo.thermal)
plt.colorbar()
plt.gca().invert_yaxis()
plt.xticks([])
plt.title('temperature [deg C]')

plt.subplot(3,1,2)
plt.scatter(dfsub['longitude'], dfsub['depth'], c=dfsub['salinity'], cmap = cmo.haline)
plt.colorbar()
plt.gca().invert_yaxis()
plt.xticks([])
plt.title('practical salinity')

plt.subplot(3,1,3)
plt.scatter(dfsub['longitude'], dfsub['depth'], c=dfsub['dissolved_oxygen'])
plt.colorbar()
plt.title('dissolved oxygen [$\mu$mol/kg]')
plt.gca().invert_yaxis()

plt.tight_layout()

#### Exercise
- Discuss the plots above with your neighbor. Describe the major features that you notice. What looks like "noise" to you?

### Set up 

First define 1-D arrays that contain the original (uninterpolated) data

In [ ]:
var = np.array(dfsub['dissolved_oxygen'])
x = np.array(dfsub['longitude'])
y = np.array(dfsub['depth'])
dive = np.array(dfsub['profile_id'])

Now define a "regular" grid that you want to interpolate to. This should have evenly spaced values. This same grid will be used for two different interpolation methods.

In [ ]:
deltax = # choose a deltax value
deltay = # choose a deltay value

xg = np.arange(x.min(), x.max(), deltax)
yg = np.arange(y.min(), y.max(), deltay)

### Method 1: griddata

First use meshgrid to create 2D arrays for the x and y grid points.

### Method 2: optimal interpolation (kriging)

Use glidertools package to make a semi-variogram. Note that we need to define an option `xy_ratio` because the data are anisotropic (the x and y data have different length scales).

In [ ]:
vargram = gt.mapping.variogram(var, x, y, dive)

In [ ]:
vargram = gt.mapping.variogram(var, x, y, dive, xy_ratio=1)

#### Exercise 
- Manually adjust the `xy_ratio` option until you can identify a well-defined "sill" in the semivariogram plot

In [ ]:
vargram

The "sill" should closely match the overall variance of the dataset. The nugget describes the "noise" level at small scales. The values are in terms of variance, or $\sigma^2$.

Using the properties of the semivariogram to define the parameters for optimal interpolation (kriging)

In [ ]:
interpolated = gt.mapping.interp_obj(
    x, y, var, xg, yg,

    # Kriging interoplation arguments
    partial_sill=5650,  # taken from the semivariogram (sill - nugget)
    nugget=200,  # taken from the semivariogram
    lenscale_x=0.41,  # in degrees if x and xg are longitude
    lenscale_y=415,  # the vertical gridding influence
)

In [ ]:
plt.figure()
plt.subplot(3, 1, 1)
plt.scatter(x, y, c=var)
plt.gca().invert_yaxis()
plt.colorbar()
xl = plt.xlim()
plt.title('original (uninterpolated) data')

plt.subplot(3, 1, 2)
plt.pcolormesh(interpolated.x, interpolated.y, interpolated.z)
plt.colorbar()
#plt.contour(interpolated.x, interpolated.y, interpolated.z, cmap='gray')
#plt.plot(x, y, 'w.', markersize=0.1)
plt.gca().invert_yaxis()
plt.xlim(xl)
plt.title('interpolated data')

plt.subplot(3, 1, 3)
plt.pcolormesh(interpolated.x, interpolated.y, interpolated.variance)
plt.plot(x, y, 'w.', markersize=0.1)
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlim(xl)
plt.title('error (variance)')
plt.tight_layout()

### Exercises

- What are the units for each subplot above?
- Choose a parameter in the kriging process above and try alternative values for it. How does it affect your interpolated map? Discuss the results of your experimentation with your neighbor.